In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
  except RuntimeError as e:
    print(e)

In [2]:
# from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.applications.densenet import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

import numpy as np
import cv2
import os

# Load Face Detector (Caffe Model)

In [3]:
prototxtPath = os.getcwd()+r"\deploy.prototxt"
weightsPath = os.getcwd()+ r"\res10_300x300_ssd_iter_140000.caffemodel"

In [4]:
net = cv2.dnn.readNet(prototxtPath, weightsPath)

# Load the trained Face Mask Detection Model

In [5]:
# model = load_model(os.getcwd()+r"/Trained_Models/mobileNetV2_facemaskmodel")
model = load_model(os.getcwd()+r"/Trained_Models/DenseNet169/Saved_Model")

In [14]:
# We are taking its height and width only not taking its channel (RGB)
# image = cv2.imread(os.getcwd() + r"/Test_Images/Test_Image_7.jpg")
image = cv2.imread(os.getcwd() + r"/Balanced_Dataset/Correct_Mask/00041_Mask.jpg")
# print(image.shape)

# Perform rescale of images if needed, comment below to remove rescaling 
scale_percent = 60 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)

# # Include border for if necessary 
# colour = [255,255,255]
# image= cv2.copyMakeBorder(image.copy(),100,100,75,75,cv2.BORDER_CONSTANT,value=colour)

# resize image
image = cv2.resize(cv2.resize(image, (300, 300)), dim, interpolation = cv2.INTER_AREA)
(h,w) = image.shape[:2]

# Face Mask Detection using Images

In [15]:
# Using BlobFromImage to feed forward the image into the face detector to detect faces
blob = cv2.dnn.blobFromImage(image,1.0, (300,300), (104.0, 177.0, 123.0),crop=False)
net.setInput(blob)
detections = net.forward()

# for faces found in the images
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    
    # if the confidence of the faces found is greater than 0.5, 
    # perform bounding boxes and include probability of 3 different classes 
    # Mask, No Mask, or Incorrect Mask
    if confidence>0.5:

        box = detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX, startY, endX, endY) = box.astype('int')
        
        # ensure the bounding boxes fall wihtin the dimensions of the frame
        (startX, startY) = (max(0,startX), max(0,startY))
        (endX, endY) = (min(w-1,endX), min(h-1,endY))
        
        # extract the face Region of Interest, convert it from BGR to RGB channel
        # resize it to 224, 224 because our model are using Tensor Input of size 224x224
        face = image[startY:endY, startX:endX]
        
        # cvtc - convert colour
        face = cv2.cvtColor(face, cv2.COLOR_BGR2RGB)
        face = cv2.resize(face, (224,224))
        
        face = img_to_array(face)
        face = preprocess_input(face)
        face = np.expand_dims(face, axis=0)
        
        (Corret_Mask, Incorrect_Mask_Chin, Incorrect_Mask_Mouth_Chin, Incorrect_Mask_Nose_Mouth, No_Mask) = model.predict(face)[0]

        # define the class label and color for the bounding boxes and text
        # in our project we are using Mask, No, Mask and Incorrect Mask
        index = (Corret_Mask, Incorrect_Mask_Chin, Incorrect_Mask_Mouth_Chin, Incorrect_Mask_Nose_Mouth, No_Mask).index(max(Corret_Mask, Incorrect_Mask_Chin, Incorrect_Mask_Mouth_Chin, Incorrect_Mask_Nose_Mouth, No_Mask))
        if index==0:
            label = "Mask "
            color = (0,255,0)
        elif index==1:
            label = "Incorrect Mask(Uncovered Nose Mouth)"
            color = (0, 165, 255)
        elif index==2:
            label = "Incorrect Mask(Uncovered Nose)"
            color = (0, 165, 255)
        elif index==3:
            label = "Incorrect Mask(Uncovered Chin)"
            color = (0, 165, 255)
        elif index==4:
            label = "No mask "
            color = (0,0,255)
        
        # include probability and label for bounding boxes
        label = label + "{prob}".format(prob = round(max(Corret_Mask, Incorrect_Mask_Chin, Incorrect_Mask_Mouth_Chin, Incorrect_Mask_Nose_Mouth, No_Mask)*100, 2))
        
        # put the text and rectange in the image
        cv2.putText(image, label, (startX,endY-10),cv2.FONT_HERSHEY_SIMPLEX, 0.45,color,2)
        cv2.rectangle(image, (startX, startY), (endX,endY),color,2)

        
# show the image, until user press 0 or e
# xit the window
cv2.imshow("Output", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
